Requires: python==3.7, pytorch==1.13.1

Download English LSTM model and vocab file from https://github.com/facebookresearch/colorlessgreenRNNs/tree/main/data

In [1]:
import util

# get model
path = '../data/model/model.pt'
model = util.load_model(path)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded model onto cpu


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'model.RNNModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has ch

In [2]:
# import sentence files and tokenize using vocab.txt

from dictionary import Dictionary

dict = Dictionary("../data/model/vocab.txt")

files = [
    "ff_unambiguous.txt",
    "fm_unambiguous.txt",
    "mf_unambiguous.txt",
    "mm_unambiguous.txt",
    "fm_stereotypical.txt",
    "mf_stereotypical.txt"
    ]

data = []
for f in files:
    path = "../data/text/" + f
    loc_path = "../data/text/locations_" + f
    print(path)

    tokens = dict.tokenize(path)

    with open(loc_path, "r") as loc:
        locations = loc.readlines()
        locations = list(map(lambda x: 
                             list(map(int, x.strip().split())), 
                             locations))
    
    data.append((f, tokens, locations))


../data/text/ff_unambiguous.txt
../data/text/fm_unambiguous.txt
../data/text/mf_unambiguous.txt
../data/text/mm_unambiguous.txt
../data/text/fm_stereotypical.txt
../data/text/mf_stereotypical.txt


In [4]:
# get gender contributions (original paper figure 4)

import gender_bias
import torch

for file, tokens, locations in data:
    bias = torch.zeros((4,))
    for ts, locs in zip(tokens, locations):
        ts = ts.unsqueeze(1)
        s1, s2, o1, o2 = locs
        bias += torch.tensor(gender_bias.get_bias(model, ts, s1, s2, o1, o2))
    print(file)
    print(bias / len(tokens))
    

ff_unambiguous.txt
tensor([0.2547, 0.3643, 0.2709, 0.3617])
fm_unambiguous.txt
tensor([-0.0089,  0.1222,  0.1563,  0.1333])
mf_unambiguous.txt
tensor([0.1247, 0.1029, 0.0292, 0.1230])
mm_unambiguous.txt
tensor([ 0.0364,  0.0052,  0.0556, -0.0215])
fm_stereotypical.txt
tensor([0.0364, 0.0304, 0.0501, 0.0501])
mf_stereotypical.txt
tensor([ 0.0567, -0.0224,  0.0349,  0.0349])


In [4]:
# get gender predictions (original paper table 2)

import gender_bias
import torch

for file, tokens, locations in data:
    predictions = torch.zeros((7,))
    for ts, locs in zip(tokens, locations):
        ts = ts.unsqueeze(1)
        s1, s2, o1, o2 = locs
        predictions += torch.Tensor(gender_bias.predict_gender(model, ts, s1, s2, o1, o2))
    print(file)
    # full, subj, subj_bias, obj, obj_bias, intercept, intercept_bias
    print(predictions / len(tokens))
    

ff_unambiguous.txt
tensor([0.2807, 0.0000, 0.9474, 0.0000, 0.9825, 1.0000, 1.0000])
fm_unambiguous.txt
tensor([0.3974, 0.0000, 0.3590, 0.8333, 0.9744, 0.9872, 0.9872])
mf_unambiguous.txt
tensor([0.3718, 0.4744, 0.9872, 0.1154, 0.6282, 0.9872, 0.9872])
mm_unambiguous.txt
tensor([1.0000, 0.7115, 1.0000, 0.9615, 1.0000, 1.0000, 1.0000])
fm_stereotypical.txt
tensor([0.8571, 0.7143, 0.8095, 1.0000, 1.0000, 1.0000, 1.0000])
mf_stereotypical.txt
tensor([0.9426, 0.8115, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
